In [1]:
#!/usr/bin/env python
# coding: utf-8
import numpy
import pygad
import pygad.nn
import pygad.gann
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def read(address):
    df = pd.read_csv(address, sep="\t",header=None)
    df.columns = ["Time", "Open", "High", "Low", "Close", "Volume"]
    return df


data = read("XAUUSD1.csv")
#data = data.iloc[:100000,:]
values = data.values
data.head(2)


window_size = 25
total = 1000
number_of_each_order = 1
fee = 0.8
vector_size = 5 * window_size

In [2]:
currents = []
features = []
for i in tqdm(range(window_size, data.shape[0])):
    history = values[i-window_size:i, 1:]
    mean = history.mean(axis=0)
    history = history - mean
    currents.append(values[i])
    data_inputs = history.reshape((1, vector_size)).astype(float)
    features.append(data_inputs)
    
features = np.concatenate(features, axis=0)

100%|██████████| 199975/199975 [00:17<00:00, 11646.60it/s]


In [3]:
def predict(prediction, close):    
    def get(value, shift):
        value = max(value, 0.0000001)
        return (( value / 100) + shift) * close        
    buy = get(prediction[0], 0.99)
    buy_sl = get(prediction[1], 0.99)
    buy_sl = min(buy_sl, buy - 1.5 * fee)
    buy_tp = get(prediction[2], 1)
    sell = get(prediction[3], 1)
    sell_sl = get(prediction[4], 1)
    sell_sl = max(sell_sl, sell + 1.5 * fee)
    sell_tp = get(prediction[5], 0.99)
    return [buy, buy_sl, buy_tp, sell, sell_sl, sell_tp]


def fitness_func(solution, sol_idx):
    buy_order = []
    sell_order = []
    profit = total
    
    global GANN_instance    
    predictions = pygad.nn.predict(last_layer=GANN_instance.population_networks[sol_idx],
                                   data_inputs=features, problem_type="regression")
    for i in range(len(currents)):    
        current = currents[i]
        high = current[2]
        low = current[3]
        close = current[4]
        [buy, buy_sl, buy_tp, sell, sell_sl, sell_tp] = predict(predictions[i], close)            
        #### for current order
        # buy
        for order in buy_order:
            price = order[0]
            sl = order[1]
            tp = order[2]
            # check sl
            if(low <= sl):
                profit -= (price - sl) + fee
                buy_order.remove(order)
            # check tp
            elif(high >= tp):
                profit += (tp - price) - fee
                buy_order.remove(order)
        # sell
        for order in sell_order:
            price = order[0]
            sl = order[1]
            tp = order[2]
            # check sl
            if((high + fee) >= sl):
                profit -= sl - price
                sell_order.remove(order)
            # check tp
            elif((low + fee) <= tp):
                profit += price - tp                
                sell_order.remove(order)
        #### for new order
        # buy
        if(low <= buy):
            if(len(buy_order) < number_of_each_order):
                buy_order.append([buy, buy_sl, buy_tp])
        # sell
        if(high >= sell):
            if(len(sell_order) < number_of_each_order):
                sell_order.append([sell, sell_sl, sell_tp])    
    return profit


def callback_generation(ga_instance):
    global GANN_instance
    population_matrices = pygad.gann.population_as_matrices(population_networks=GANN_instance.population_networks,
                                                            population_vectors=ga_instance.population)
    GANN_instance.update_population_trained_weights(population_trained_weights=population_matrices)
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Accuracy   = {fitness}".format(fitness=ga_instance.best_solution()[1]))



GANN_instance = pygad.gann.GANN(num_solutions=25,
                                num_neurons_input=vector_size,
                                num_neurons_hidden_layers=[32, 16],
                                num_neurons_output=6,
                                hidden_activations=["relu", "relu"],
                                output_activation="sigmoid")


population_vectors = pygad.gann.population_as_vectors(population_networks=GANN_instance.population_networks)


ga_instance = pygad.GA(num_generations=50,
                       num_parents_mating=10,
                       initial_population=population_vectors.copy(),
                       fitness_func=fitness_func,
                       mutation_percent_genes=5,
                       callback_generation=callback_generation)

ga_instance.run()

ga_instance.plot_result()


solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution)
print(solution_fitness)
print(solution_idx)

/home/emadi/.local/lib/python3.8/site-packages/pygad/pygad.py:860: UserWarning: Starting from PyGAD 2.6.0, the callback_generation parameter is deprecated and will be removed in a later release of PyGAD. Please use the on_generation parameter instead.
  if not self.suppress_warnings: warnings.warn("Starting from PyGAD 2.6.0, the callback_generation parameter is deprecated and will be removed in a later release of PyGAD. Please use the on_generation parameter instead.")


Generation = 1


KeyboardInterrupt: 